In [2]:
import os
import numpy as np
import pandas as pd
from scipy import interpolate

In [45]:
!ls ../../data/group15

139_Run000260_000000.txt


In [12]:
data = pd.read_csv('../../data/Run000260/data_000000.txt')

#adding time column
data['t'] = data['BX_COUNTER']*25 + data['TDC_MEAS']*25/30


#List of conditions to assign a hit to a detector
fpga = data['FPGA'] == 1 
channel = (data['TDC_CHANNEL'] > 64) & (data['TDC_CHANNEL'] <= 128) #condition on TDC_CHANNEL
trigger = data['TDC_CHANNEL'] >= 129 

conditions = [trigger, ~fpga & ~channel, ~fpga & channel, fpga & ~channel, fpga & channel]
values = [0, 1, 2, 3, 4]

#adding detector column
data['DETECTOR'] = np.select(conditions, values)


#List of conditions to assign a hit to a layer
layer_4 = data['TDC_CHANNEL']%4 == 0 
layer_3 = (data['TDC_CHANNEL'] + 2) % 4 == 0
layer_2 = (data['TDC_CHANNEL'] + 1 ) % 4 == 0
layer_1 = (data['TDC_CHANNEL'] + 3) % 4 == 0
trigger = data['TDC_CHANNEL'] >= 129 

#adding layer column
conditions = [trigger, layer_4, layer_3, layer_2, layer_1]
values = [0, 4, 3, 2, 1]

data['LAYER'] = np.select(conditions, values)

#add column coordinate of the cell
data['COLUMN'] = ((data['TDC_CHANNEL'] % 64 
                   - interpolate.BarycentricInterpolator([1,2,3,4],[1,3,2,4])(data['LAYER'])) / 4 + 1).astype(int)

#set column value to 0 when trigger is activate
data.loc[data['DETECTOR'] == 0, 'COLUMN'] = 0

In [47]:
#i consider orbit only with trigger, i drop other trigger raws
data_139 = data.groupby('ORBIT_CNT', group_keys=False).filter(lambda x: 139 in x.TDC_CHANNEL.values)
data_139 = data_139[data_139['TDC_CHANNEL'] != 130]
data_139 = data_139[data_139['TDC_CHANNEL'] != 129]
data_139 = data_139[data_139['TDC_CHANNEL'] != 137]
data_139 = data_139[data_139['TDC_CHANNEL'] != 138]

In [48]:
#identify mutiple events inside the same orbit

def Identify_Events(x):
    
    if (x.t.max() - x.t.min()) < 415:
    
        x['EVENT'] = 1
        return x
    
    else:
        
        t1 = x.t.iloc[0]
        x1 =  x[np.abs(x.t.values - t1) < 415]
        x1['EVENT'] = 1
        
        xc = x[np.abs(x.t.values - t1) >= 415]
        i = 1
        while(xc.size != 0):
            
            i += 1
            t2 = xc.t.iloc[0]
            x2 =  xc[np.abs(xc.t.values - t2) < 415]
            x2['EVENT'] = i
            x1 = pd.concat([x1,x2])
            xc = xc[np.abs(xc.t.values - t2) >= 415]
        
        return x1

In [49]:
#define the function that for esch orbit return the orbit with the values that have difference with the time of the 
#TDC_channel 139 less than the T_max 
#ridondance in the triggers (the sames in different ambiguity), will be solved later
#computing the drift time

def Time_Max_Filer(x):
        
        if (x.DETECTOR==0).sum() == 0:
            return None
            
            
        if (x.DETECTOR==0).sum() == 1:
        
            trigger_time = x[x.DETECTOR==0].t.values.min()
            x1 =  x[(x.t.values - trigger_time < 430)*(x.t.values - trigger_time > 0)]
            x1['DRIFT_TIME'] = x1.t.values - trigger_time
            
            return x1

In [50]:
#adding events column
data_139 = data_139.groupby('ORBIT_CNT', group_keys=False).apply(Identify_Events)

In [51]:
#dropping events with less than 2 cells not counting the trigger 
data_139 = data_139.groupby(['ORBIT_CNT', 'EVENT'], group_keys=False).apply(Time_Max_Filer)

In [52]:
#dropping trigger events
data_139 = data_139[data_139['DETECTOR'] != 0]

In [53]:
#dropping events with number of hits hit less than 3 (ambiguity nont solvable)
data_139 = data_139.groupby(['ORBIT_CNT', 'EVENT', 'DETECTOR']).filter(lambda x: len(x) >= 3)

In [54]:
#dropping some columns
data_139 = data_139.drop(['HEAD', 'FPGA', 'TDC_CHANNEL', 'BX_COUNTER', 'TDC_MEAS'], axis = 1)

In [79]:
data_139 = data_139.groupby(['ORBIT_CNT','EVENT','DETECTOR']).filter(lambda x: np.unique(x.LAYER.values).size >= 3)

In [80]:
data_139.groupby(['ORBIT_CNT','EVENT', 'DETECTOR', 'LAYER', 'COLUMN', 'DRIFT_TIME']).mean()

t
ORBIT_CNT  EVENT DETECTOR LAYER COLUMN DRIFT_TIME               
1897414934 1     1        1     8      -42.500000   50657.500000
                                9      -38.333333   50661.666667
                          2     7      -90.833333   50609.166667
                          3     7      -333.333333  50366.666667
                                       -39.166667   50660.833333
                          4     6      -339.166667  50360.833333
1897415301 1     2        1     7      -102.500000  70197.500000
                          2     6      -342.500000  69957.500000
                          3     6      -227.500000  70072.500000
                          4     6      -30.000000   70270.000000
1897415425 1     3        1     9      -30.833333   33344.166667
                          2     8      -298.333333  33076.666667
                          3     9      -143.333333  33231.666667
                          4     8      -186.666667  33188.333333
1897415544 1     4        1     8      -204.166667  30645.833333
                          2     7      -199.166667  30650.833333
                          3     8      -27.500000   30822.500000
                          4     7      -346.666667  30503.333333
1897415674 1     3        1     6      -32.500000   29917.500000
                                11     -67.500000   29882.500000
                          2     10     -99.166667   29850.833333
                          3     10     -107.500000  29842.500000
                          4     9      -43.333333   29906.666667
1897416153 1     3        1     3      -323.333333  73051.666667
                          2     2      -28.333333   73346.666667
                          3     3      -96.666667   73278.333333
                          4     2      -328.333333  73046.666667
1897416210 1     2        1     6      -122.500000  73652.500000
                                7      -241.666667  73533.333333
                          2     6      -25.833333   73749.166667
...                                                          ...
1920940234 1     3        1     6       269.166667  47994.166667
                          2     3       220.000000  47945.000000
                                5      -60.833333   47664.166667
                          3     4      -27.500000   47697.500000
                                6      -47.500000   47677.500000
                                        266.666667  47991.666667
                          4     3       245.833333  47970.833333
                                5      -75.000000   47650.000000
                                        210.000000  47935.000000
                 4        1     5      -43.333333   47681.666667
                          2     4       221.666667  47946.666667
                                5       241.666667  47966.666667
                          3     5      -69.166667   47655.833333
                          4     4       249.166667  47974.166667
                                5       259.166667  47984.166667
1920940237 1     2        1     14     -34.166667     315.833333
                          2     13     -148.333333    201.666667
                          3     14     -40.833333     309.166667
                          4     13     -137.500000    212.500000
1920940252 1     3        1     7      -48.333333   13651.666667
                          2     6       71.666667   13771.666667
                          3     7      -57.500000   13642.500000
                          4     6       66.666667   13766.666667
                 4        1     7      -109.166667  13590.833333
                          2     6       118.333333  13818.333333
                          3     7      -100.833333  13599.166667
                          4     6       118.333333  13818.333333
1920940255 1     3        1     10     -20.833333   62829.166667
                          2     9      -36.666667   62813.333333
                          3     1

In [41]:
data_139.to_csv('../../data/group15/139_Run000260_000000.txt', index = False)

In [ ]:
#ignorare quello che c'è sotto

In [10]:
data_139prova = pd.read_csv('../../data/group15/139_Run000260_000000.txt')

In [11]:
data_139prova.groupby(['ORBIT_CNT','EVENT', 'DETECTOR', 'LAYER', 'COLUMN', 'DRIFT_TIME']).mean()

t
ORBIT_CNT  EVENT DETECTOR LAYER COLUMN DRIFT_TIME              
1897417046 1     4        2     12     110.000000  36860.000000
                          3     8      30.833333   36780.833333
                          4     13     95.000000   36845.000000
1897418311 1     4        1     10     9.166667     1184.166667
                          3     9      35.833333    1210.833333
                          4     8      60.000000    1235.000000
                                10     26.666667    1201.666667
1897431396 1     3        1     16     17.500000   15317.500000
                                       204.166667  15504.166667
                          2     14     16.666667   15316.666667
                                       145.833333  15445.833333
                          3     13     110.000000  15410.000000
1897433139 1     2        1     10     216.666667  37666.666667
                                11     131.666667  37581.666667
                          4     14     135.000000  37585.000000
1897434251 1     3        1     5      264.166667  54789.166667
                                6      182.500000  54707.500000
                          2     5      89.166667   54614.166667
1897436757 2     3        1     8      251.666667   7826.666667
                                9      178.333333   7753.333333
                                14     62.500000    7637.500000
                          2     8      37.500000    7612.500000
                                14     154.166667   7729.166667
                                15     240.833333   7815.833333
                          3     11     215.833333   7790.833333
                                12     257.500000   7832.500000
                                14     168.333333   7743.333333
                                16     96.666667    7671.666667
                          4     9      115.833333   7690.833333
1897439718 1     1        1     1      175.000000  16775.000000
...                                                         ...
1920939994 1     3        1     4      302.500000  53377.500000
                          2     5      83.333333   53158.333333
                                       212.500000  53287.500000
                                6      81.666667   53156.666667
                          3     4      250.833333  53325.833333
                                5      0.833333    53075.833333
                                7      165.833333  53240.833333
                                11     229.166667  53304.166667
                          4     3      31.666667   53106.666667
                                4      183.333333  53258.333333
                                6      74.166667   53149.166667
                                10     117.500000  53192.500000
                 4        1     3      10.833333   53085.833333
                                7      185.833333  53260.833333
                          2     3      225.000000  53300.000000
                                6      297.500000  53372.500000
                                16     35.833333   53110.833333
                          3     3      115.833333  53190.833333
                                6      264.166667  53339.166667
                          4     5      7.500000    53082.500000
                                6      7.500000    53082.500000
1920940234 1     3        1     6      269.166667  47994.166667
                          2     3      220.000000  47945.000000
                          3     6      266.666667  47991.666667
                          4     3      245.833333  47970.833333
                                5      210.000000  47935.000000
                 4        2     4      221.666667  47946.666667
                                5      241.666667  47966.666667
                          4     4      249.166667  47974.166667
                                5      259.166667  47984.166667

[65807 rows x 1 columns]

In [ ]:
#da sotto in poi sono cose da continuare

In [30]:
def close(layer1, column1, layer2, column2):
    
    if (layer1 == layer2) and np.abs(column1 - column2) <= 1:
        return True
    
    if layer1%2 == 1 and layer2 == layer1 + 1:
        if (column2 == column1) or (column2 == column1 + 1):
            return True
        
    if layer1%2 == 0 and layer2 == layer1 + 1:
        if (column2 == column1 - 1) or (column2 == column1):
            return True
        
    if layer2 == layer1 + 2:
        if np.abs(column2 - column1) <= 1:
            return True
    
    
    return False

In [31]:
close(1,2,3,4)

False

In [33]:
data_139prova.groupby('ORBIT_CNT').get_group(1920939994).sort_values('LAYER')

,ORBIT_CNT,t,DETECTOR,LAYER,COLUMN,EVENT,DRIFT_TIME
65793,1920939994,53260.833333,4,1,7,1,185.833333
65783,1920939994,53085.833333,4,1,3,1,10.833333
65796,1920939994,53377.500000,3,1,4,1,302.500000
65797,1920939994,53372.500000,4,2,6,1,297.500000
65778,1920939994,53110.833333,4,2,16,1,35.833333
65794,1920939994,53287.500000,3,2,5,1,212.500000
65781,1920939994,53158.333333,3,2,5,1,83.333333
65791,1920939994,53300.000000,4,2,3,1,225.000000
65788,1920939994,53156.666667,3,2,6,1,81.666667
65795,1920939994,53339.166667,4,3,6,1,264.166667
